In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving covid.csv to covid.csv


In [ ]:
import io
data=pd.read_csv(io.BytesIO(uploaded['covid.csv']))
data.head()

,Sno,age,gender,body temperature,Dry Cough,sour throat,weakness,breathing problem,drowsiness,pain in chest,travel history to infected countries,diabetes,heart disease,lung disease,stroke or reduced immunity,symptoms progressed,high blood pressue,kidney disease,change in appetide,Loss of sense of smell,Corona result
0,1,20,1,98.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,19,1,99.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,55,0,102.0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1
3,4,40,0,100.0,0,0,0,0,1,1,1,1,0,0,0,1,1,0,1,0,2
4,5,33,1,99.2,0,1,0,1,0,0,1,0,0,1,1,1,0,0,0,1,2


In [ ]:
data.describe()

,Sno,age,gender,body temperature,Dry Cough,sour throat,weakness,breathing problem,drowsiness,pain in chest,travel history to infected countries,diabetes,heart disease,lung disease,stroke or reduced immunity,symptoms progressed,high blood pressue,kidney disease,change in appetide,Loss of sense of smell,Corona result
count,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000
mean,64.000000,36.456693,0.527559,100.031496,0.488189,0.551181,0.409449,0.433071,0.425197,0.370079,0.464567,0.244094,0.094488,0.141732,0.165354,0.480315,0.283465,0.133858,0.401575,0.448819,0.937008
std,36.805797,14.052484,0.501217,1.569536,0.501840,0.499343,0.493680,0.497463,0.496331,0.484738,0.500718,0.431250,0.293665,0.350157,0.372971,0.501591,0.452465,0.341848,0.492158,0.499343,0.675513
min,1.000000,18.000000,0.000000,96.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.500000,25.500000,0.000000,98.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,64.000000,33.000000,1.000000,99.700000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,95.500000,45.000000,1.000000,101.400000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,127.000000,80.000000,1.000000,103.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [ ]:
data.isnull().values.any()

False

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

In [ ]:
data[['age','body temperature']]=mms.fit_transform(data[['age','body temperature']])

In [ ]:
data.head()

,Sno,age,gender,body temperature,Dry Cough,sour throat,weakness,breathing problem,drowsiness,pain in chest,travel history to infected countries,diabetes,heart disease,lung disease,stroke or reduced immunity,symptoms progressed,high blood pressue,kidney disease,change in appetide,Loss of sense of smell,Corona result
0,1,0.032258,1,0.366197,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0.016129,1,0.422535,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0.596774,0,0.845070,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1
3,4,0.354839,0,0.563380,0,0,0,0,1,1,1,1,0,0,0,1,1,0,1,0,2
4,5,0.241935,1,0.450704,0,1,0,1,0,0,1,0,0,1,1,1,0,0,0,1,2


In [ ]:
del data['Sno']

In [ ]:
x=data.values[:,0:19]
y=data.values[:,19]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15)

In [ ]:
from sklearn.naive_bayes import BernoulliNB 
mnb = BernoulliNB()
g = mnb.fit(x_train, y_train)
y_pred = g.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred)

0.65

In [ ]:
from sklearn import ensemble
gb=ensemble.GradientBoostingClassifier()
gb.fit(x_train, y_train)
gb.score(x_train, y_train)

0.9813084112149533

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc=gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
wb=gc.open_by_url('https://docs.google.com/spreadsheets/d/14upVX8lk9DABhNPMfTQKMrK8ls8Xd3SpA9DbDyL4Tpk/edit#gid=0')

In [ ]:
sheet=wb.worksheet('Sheet1')

In [ ]:
a=sheet.get_all_values()

In [ ]:
def listfromsheet(a):
  for i in range(len(a)):
    if i==len(a)-1:
      print(len(a[i]))
      return(a[i])


In [ ]:
newlist=listfromsheet(a)
newlist=newlist[0:19]
newlist=list(map(float,newlist))
newlist=np.resize(newlist,(1,19))
y_pred=gb.predict(newlist)

def output(y_pred):
    
    if y_pred==0:
        return('Mildly Severe')
    elif y_pred==1:
        return('Moderately Severe')
    else:
        return('Highly Severe')

severity=output(y_pred)
print(severity)

20
Highly Severe


In [ ]:
sheet.update_cell(len(a),20,severity)

{'spreadsheetId': '14upVX8lk9DABhNPMfTQKMrK8ls8Xd3SpA9DbDyL4Tpk',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': 'Sheet1!T4',
 'updatedRows': 1}